***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5530  GCE       4                                       RUNNING  us-central1-f


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install requests
!pip install lxml
!pip install bs4

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 14:04 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ex3ir205557564' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith(".parquet"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd

In [9]:
# parquetFile.show()

In [10]:
# parquetFile.createOrReplaceTempView("parquetFile")
# spark.sql("SELECT title from parquetFile WHERE id = 20469797").rdd.collect()

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [11]:
# Count number of wiki pages
parquetFile.select("title", "id").count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [12]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [13]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [14]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [25]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
dl = []

def calc_dl(text, wiki_id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token for token in tokens if not token in all_stopwords]
    return (wiki_id, len(tokens))
    
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    # YOUR CODE HERE
    tokens = [token for token in tokens if not token in all_stopwords]
    c:Counter = Counter(tokens)
    tuples = []
    for word, count in c.items():
        tuples.append((word, (id, count)))
    return tuples

def reduce_word_counts(unsorted_pl):
      ''' Returns a sorted posting list by wiki_id.
      Parameters:
      -----------
        unsorted_pl: list of tuples
          A list of (wiki_id, tf) tuples 
      Returns:
      --------
        list of tuples
          A sorted posting list.
      '''
      # YOUR CODE HERE
      return sorted(unsorted_pl, key= lambda x:x[0])

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.
    '''
    # YOUR CODE HERE
    return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings, name):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
    '''
    # YOUR CODE HERE
    rdd = postings.groupBy(lambda x: f"{token2bucket_id(x[0])}")
    rdd= rdd.map(lambda x: InvertedIndex.write_a_posting_list(x, "ex3ir205557564", name))
    return rdd

In [26]:
# time the index creation time
def create_index(doc_pairs, name, with_filter):
    t_start = time()
    # word counts map
    word_counts = doc_pairs.flatMap(lambda x: word_count(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    # filtering postings and calculate df
    postings_filtered = postings.filter(lambda x: len(x[1])>50) if with_filter else postings
#     partition_postings_and_write(postings_filtered, name).take(1)
#     print(postings_filtered.take(1).collect())
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    # partition posting lists and write out
    _ = partition_postings_and_write(postings_filtered, name).collect()
    index_const_time = time() - t_start
    print(f"{name}: {index_const_time}")
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f'{name}_postings_gcp'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index('.', f'{name}_index')
    # upload to gs
    index_src = f"{name}_index.pkl"
    index_dst = f'gs://{bucket_name}/{name}_postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    !gsutil ls -lh $index_dst
    return inverted



In [27]:
rdd_body = doc_text_pairs.map(lambda x: calc_dl(x[0], x[1]))
rdd_title = doc_title_pairs.map(lambda x: calc_dl(x[0], x[1]))
rdd = rdd_body.union(rdd_title)
from operator import add
collect = rdd.foldByKey(0, add).collect()
wiki_ids, lengths = zip(*collect)

from inverted_index_gcp import write_dl, read_dl
write_dl("dl_test/wiki_ids.bin", wiki_ids,15,2)
write_dl("dl_test/lengths.bin", lengths,7,1)

In [31]:
print(len(wiki_ids))
limit = 2**8
# print(len(read_dl("dl_test/wiki_ids.bin", len(wiki_ids))))
print(len([i for i in lengths if i<limit]))
print([idd for i, idd in enumerate(sorted(lengths, reverse=True)) if i<1000])

6348910
4278727
[42489, 39463, 38155, 37563, 35627, 35533, 35133, 34639, 34053, 33759, 33602, 33473, 33407, 33158, 33051, 32470, 32216, 31632, 31198, 30253, 30208, 30077, 30016, 29410, 28774, 28347, 28083, 27997, 27972, 27900, 27278, 27076, 26764, 26613, 26020, 25968, 25945, 25941, 25678, 25559, 25529, 25523, 25162, 25094, 24962, 24890, 24845, 24785, 24669, 24631, 24555, 24493, 24385, 24153, 24109, 23980, 23964, 23938, 23929, 23926, 23819, 23738, 23734, 23705, 23698, 23554, 23541, 23539, 23442, 23439, 23342, 23322, 23288, 23284, 23269, 23264, 23208, 23185, 23105, 23045, 22969, 22913, 22790, 22715, 22699, 22636, 22573, 22519, 22493, 22394, 22365, 22360, 22312, 22177, 22161, 22068, 22026, 21982, 21956, 21801, 21756, 21750, 21691, 21589, 21558, 21551, 21548, 21545, 21459, 21389, 21340, 21292, 21262, 21210, 21203, 21193, 21160, 21158, 21147, 21104, 21024, 21014, 21012, 20936, 20880, 20839, 20812, 20650, 20644, 20617, 20583, 20540, 20467, 20434, 20296, 20296, 20193, 20085, 20079, 19943, 199

In [35]:
t_start = time()
d1 = read_dl("dl_test/lengths.bin", len(collect),7,1)
d2 = read_dl("dl_test/wiki_ids.bin", len(collect),15,2)
dic = {x: y for x,y in list(map(lambda x,y: (x,y), d1,d2))}
print((time()-t_start))

18.07412815093994


In [32]:
print(len([i for i in d1 if i<limit]))
print([id for i, id in enumerate(sorted(d1, reverse=True)) if i<1000])

4278727
[42489, 39463, 38155, 37563, 35627, 35533, 35133, 34639, 34053, 33759, 33602, 33473, 33407, 33158, 33051, 32470, 32216, 31632, 31198, 30253, 30208, 30077, 30016, 29410, 28774, 28347, 28083, 27997, 27972, 27900, 27278, 27076, 26764, 26613, 26020, 25968, 25945, 25941, 25678, 25559, 25529, 25523, 25162, 25094, 24962, 24890, 24845, 24785, 24669, 24631, 24555, 24493, 24385, 24153, 24109, 23980, 23964, 23938, 23929, 23926, 23819, 23738, 23734, 23705, 23698, 23554, 23541, 23539, 23442, 23439, 23342, 23322, 23288, 23284, 23269, 23264, 23208, 23185, 23105, 23045, 22969, 22913, 22790, 22715, 22699, 22636, 22573, 22519, 22493, 22394, 22365, 22360, 22312, 22177, 22161, 22068, 22026, 21982, 21956, 21801, 21756, 21750, 21691, 21589, 21558, 21551, 21548, 21545, 21459, 21389, 21340, 21292, 21262, 21210, 21203, 21193, 21160, 21158, 21147, 21104, 21024, 21014, 21012, 20936, 20880, 20839, 20812, 20650, 20644, 20617, 20583, 20540, 20467, 20434, 20296, 20296, 20193, 20085, 20079, 19943, 19920, 1985

In [ ]:
inverted_body = create_index(doc_text_pairs, "body", True)

In [ ]:
inverted_index = create_index(doc_title_pairs, "title", False)

Putting it all together

In [ ]:
body_index = InvertedIndex.read_index("/home/dataproc", "body_index")

In [ ]:
title_index = InvertedIndex.read_index("/home/dataproc", "title_index")

In [ ]:
# print(title_index.df)
print(body_index.read_term_pl("election", "body_postings_gcp"))

In [ ]:
# inverted_body.posting_locs

In [ ]:
print(title_index.read_term_pl("election", "title_postings_gcp"))

In [ ]:
# inverted_body.df